In [32]:
import csv
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os

In [6]:
badwords = ['nigger', 'fuck', 'sex', 'negro', 'cunt', 'bitch', 'cum', 'gay', 'dyke', 'fag', 'faggot']
redditwords = ['Edit:', 'edit:']


def check_bad_word(badwordlist, sentence):
    sentencecopy = sentence.split(' ')
    for word in badwords:
        if word in sentencecopy:
            return False
    return True


In [79]:
def filter_csv_indiv(data, length_limit=250, is_qa = False):
    data = data.drop(["Unnamed: 0"], axis=1)
    is_good_list = []
    for i in range(len(data)):
        title = data.iloc[i]['title']
        body = data.iloc[i]['selftext']
        sentence = title + body
        data.at[i,'title'] = title.replace('\n','')
        data.at[i,'body'] = body.replace('\n','')
        for word in redditwords:
            if word in body:
                data.at[i,'body'] = body[:body.index(word)]
        is_good = check_bad_word(badwords, sentence) and (len(body) > 2) and (len(sentence) < length_limit) and not body.eq("[deleted]") 
        if is_qa:
            is_good = is_good and sentence.count('?') == 1

        is_good_list.append(is_good)
    data['good'] = is_good_list
    data = data[data['good']]
    data = data.drop(['good'], axis=1)
    return data

In [128]:
def filter_csv2(data,prev, length_limit=250, is_qa = False):
    is_good_list = []
    for i in range(len(data)):
        title = data.iloc[i]['title']
        body = data.iloc[i]['selftext']
        if type(body) is not str or  type(title) is not str:
            continue
        sentence = title + body
        data.at[i,'title'] = title.replace('\n','')
        data.at[i,'selftext'] = body.replace('\n','')
        for word in redditwords:
            if word in body:
                data.at[i,'selftext'] = body[:body.index(word)]
        is_good = check_bad_word(badwords, sentence) and (len(sentence) > 2) and (len(sentence) < length_limit) and (title not in prev['title']) and  not body == "[deleted]" and not body =="[removed]" 
        if is_qa:
            is_good = is_good and sentence.count('?') == 1

        if is_good:
            prev=prev.append(data.loc[i,:])
    return prev

In [137]:
def write_csv(posts, name):
    name = name + ".csv"
    posts.to_csv(name)

In [20]:
def read_in_csv(csv_file):
    if '.csv' in csv_file:
        data = pd.read_csv(csv_file)
    elif '.tsv' in csv_file:
        data = pd.read_csv(csv_file,sep="\t")
    return data

In [62]:
files = os.listdir("../raw")
files = [f for f in files if (".csv" in f and "humor_challenge_jokes" not in f)]

In [132]:
pd_list = []
#csv_list = ['puns.csv', 'AntiAntiJokes.csv','humor_challenge_jokes.csv','rjokes.csv','qajokes_cleaned.csv']
giant_pd = pd.DataFrame(columns = ['score','title','selftext'])
for file in files:
    fname = "../raw/" + file
    df = read_in_csv(fname)
    if "Unnamed: 0" in df.columns:
        df = df.drop(["Unnamed: 0"], axis = 1)
    if "score" not in df.columns:
        df['score'] = np.zeros(len(df))
    if " selftext" in df.columns:
        df = df.rename(columns={" selftext": "selftext"})
    if "title" not in df.columns:
        if "selftext" in df.columns:
            df=df.rename(columns={"selftext": "title", "body": "selftext", "score":"score"})
        else:
            continue
    pd_list.append(df)

In [134]:
merged = pd.concat([pd_list[0],pd_list[14]])
del pd_list[14]

In [136]:
for p in pd_list:
    giant = filter_csv2(p,giant)

In [131]:
pd_list[4].head()

,title,selftext,score
0,Yett,I don't like 420 memes out of 20 points I give...,0
1,Titles are weird,"If I watch inspector calls, in glasses. could...",0
2,Savege answer to the officer,Officer: your headlight is broken\nMe: no your...,0
3,Quasimodo's home destroyed by fire,We will have to get over this hump somehow.,0
4,Quasimodo's home destroyed by fire,Global News: Notre Dame at risk of collapse as...,0


In [138]:
write_csv(giant,"aggregated")